In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import requests
import math
from bs4 import BeautifulSoup
import matplotlib as mlp
import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg
import matplotlib.figure
import seaborn as sb
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
#PRICES SCRAPING

In [2]:
fst_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/').content, 'lxml')
fst_page_listings_soup

In [ ]:
fst_page_prices_soup = fst_page_listings_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
fst_page_prices_soup

In [ ]:
fst_page_prices = re.findall('[0-9]+,[0-9]+', str(fst_page_prices_soup))
fst_page_prices

In [ ]:
#ANOTHER WAY TO SCRAPE THE PRICES
#listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=2').content, 'lxml')
#listings_soup.find_all('h4', {'class': 'heading-3 margin-y-1 font-weight-bold'})

In [ ]:
#Scraping rest of pages listings prices
def pricesscraper(number):
    nth_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number)).content, 'lxml')
    nth_page_prices_soup = nth_page_listings_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
    nth_page_prices = re.findall('[0-9]+,[0-9]+', str(nth_page_prices_soup))
    return nth_page_prices

In [ ]:
prices = list(map(pricesscraper, range(2, 600)))
len(prices)

In [ ]:
prices = str(prices).replace('[', '').replace(']', '').split(', ')
prices

In [ ]:
prices = list(map(lambda x: x[1:-1], prices))
prices

In [ ]:
prices += fst_page_prices
print(len(prices))
prices

In [ ]:
# YEARS SCRAPING

In [ ]:
fst_page_years_soup = fst_page_listings_soup.find_all('span', {'class': 'vehicle-card-year'})
fst_page_years = list(re.findall('[12][0-9]{3}', str(fst_page_years_soup)))
print(len(fst_page_years))
fst_page_years

In [ ]:
def yearsscraper(number):
    nth_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number)).content, 'lxml')
    nth_page_years_soup = nth_page_listings_soup.find_all('span', {'class': 'vehicle-card-year'})
    nth_page_years = re.findall('[12][0-9]{3}', str(nth_page_years_soup))
    return nth_page_years

In [ ]:
years = list(map(yearsscraper, range(2, 600)))
years = str(years).replace('[', '').replace(']', '').split(', ')
years = list(map(lambda x: x[1:-1], years))
years += fst_page_years
print(len(years))
years

In [ ]:
#MILEAGES SCRAPING

In [ ]:
fst_page_mileages_soup = fst_page_listings_soup.find_all('div', {'class': 'font-size-1 text-truncate'})
fst_page_mileages_soup

In [ ]:
fst_page_mileages_unf = re.findall('g>[0-9]*,[0-9]+', str(fst_page_mileages_soup))
fst_page_mileages_unf

In [ ]:
fst_page_mileage = list(map(lambda mileage: mileage[2:], fst_page_miles))
fst_page_mileage

In [ ]:
def mileagesscraper(number):
    nth_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number)).content, 'lxml')
    nth_page_listings = nth_page_listings_soup.find_all('div', {'class': 'font-size-1 text-truncate'})
    nth_page_mileages_unf = re.findall('g>[0-9]*,[0-9]+', str(nth_page_listings))
    nth_page_mileages = list(map(lambda mileage: mileage[2:], nth_page_mileages_unf))
    return nth_page_mileages

In [ ]:
mileages = list(map(mileagesscraper, range(2,600)))
mileages = str(mileages).replace('[', '').replace(']', '').split(', ')
mileages = list(map(lambda x: x[1:-1], mileages))
mileages += fst_page_mileages
print(len(mileages))
mileages

In [ ]:
#LOCATIONS STATES SCRAPING

In [ ]:
fst_page_loc_states_soup = fst_page_listings_soup.find_all('div', {'data-test': 'vehicleCardLocation'})
fst_page_loc_states = re.findall('[A-Z]{2}', str(fst_page_loc_states_soup))
fst_page_loc_states

In [ ]:
def statesscraper(number_of_page):
    nth_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number_of_page)).content, 'lxml')
    nth_page_loc_states_soup = nth_page_listings_soup.find_all('div', {'data-test': 'vehicleCardLocation'})
    nth_page_loc_states = re.findall('[A-Z]{2}', str(nth_page_loc_states_soup)) 
    return nth_page_loc_states

In [ ]:
states = list(map(statesscraper, range(2, 600)))
states = str(prices).replace('[', '').replace(']', '').split(', ')
states = list(map(lambda x: x[1:-1], states))
states += fst_page_loc_states
print(len(states))
states

In [76]:
#features scraping function
def scraper(tag, element, element_description, number_of_page, regex):
    fst_page_features_soup = fst_page_listings_soup.find_all(tag, {element: element_description})
    fst_page_features = re.findall(regex, str(fst_page_features_soup))
    def featuresscraper(number_of_page):
        nth_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number_of_page)).content, 'lxml')
        nth_page_features_soup = nth_page_listings_soup.find_all(tag, {element: element_description})
        nth_page_features = re.findall(regex, str(nth_page_features_soup))
        return nth_page_features
    features = list(map(lambda f: f[1: -1], str(list(map(featuresscraper, range(2, 600)))).replace('[', '').replace(']', '').split(', ')))
    features += fst_page_features
    return features

In [ ]:
cities = scraper()

In [64]:
xxxx = fst_page_listings_soup.find_all('div', {'data-test': 'vehicleCardLocation'})
xxxx[20]

<div class="vehicle-card-location font-size-1 margin-top-1" data-qa="Location" data-test="vehicleCardLocation"><svg class="icon icon-before vehicle-card-icon icon-color-default" data-qa="IconLocationPin" style="width:16px;height:16px;stroke-width:1.5" viewbox="0 0 24 24"><path d="M18.91 12.33L12.62 23 6.23 12.2"></path><path d="M6.44 12.55a7.5 7.5 0 0 1-1.35-3.93v-.39a7.54 7.54 0 0 1 15.08 0v.69a7.5 7.5 0 0 1-1.37 3.68"></path><circle cx="12.57" cy="8.09" r="2.7"></circle></svg>Inver Grove Heights<!-- -->, <!-- -->MN</div>

In [55]:
len(re.findall('>[A-Z][a-z]+<', str(xxxx)))

27

In [53]:
re.findall('>[A-Z][a-z]+[. ]+[A-z]+<!', str(xxxx))

['>Point Pleasant<!',
 '>St. Petersburg<!',
 '>San Antonio<!',
 '>Merritt Island<!',
 '>Fort Myers<!']

In [75]:
re.findall('>[A-Z][a-z]+[. ]*[A-z]*[ ]*[A-z]*<', str(xxxx))

['>Stafford<',
 '>Carrollton<',
 '>Point Pleasant<',
 '>St. Petersburg<',
 '>Boulder<',
 '>Lumberton<',
 '>Kingsport<',
 '>Gorham<',
 '>Denver<',
 '>Hollywood<',
 '>Blair<',
 '>San Antonio<',
 '>Puyallup<',
 '>Hamilton<',
 '>Modesto<',
 '>Hollywood<',
 '>Shelby<',
 '>Belton<',
 '>Hanford<',
 '>Charlotte<',
 '>Inver Grove Heights<',
 '>Merritt Island<',
 '>Margate<',
 '>Aberdeen<',
 '>Sacramento<',
 '>Caledonia<',
 '>Raleigh<',
 '>Wilson<',
 '>Boise<',
 '>Bloomington<',
 '>Fontana<',
 '>Fort Myers<',
 '>Austin<']

In [78]:
cities = scraper('div', 'data-test', 'vehicleCardLocation', 10, '>[A-Z][a-z]+[. ]*[A-z]*[ ]*[A-z]*<')
len(cities)

17919

In [79]:
cities

['>Minot<',
 '>Mesa<',
 '>Minot<',
 '>Sewell<',
 '>Riverside<',
 '>Shelby<',
 '>Winchester<',
 '>Chester<',
 '>Sacramento<',
 '>North Riverside<',
 '>Miami<',
 '>Columbia<',
 '>Jacksonville<',
 '>Kaktovik<',
 '>Siloam Springs<',
 '>Greenwood<',
 '>Moreno Valley<',
 '>Jefferson City<',
 '>Seymour<',
 '>Omaha<',
 '>Broken Arrow<',
 '>Medford<',
 '>Springfield<',
 '>Roswell<',
 '>Hollywood<',
 '>Shakopee<',
 '>Lynnwood<',
 '>Duluth<',
 '>Florence<',
 '>Opa Locka<',
 '>Blue Mound<',
 '>Dallas<',
 '>Merritt Island<',
 '>Ventura<',
 '>Duluth<',
 '>Delaware<',
 '>Inver Grove<',
 '>Smyrna<',
 '>Port Arthur<',
 '>Hollywood<',
 '>Bergenfield<',
 '>Greeley<',
 '>Olathe<',
 '>Fairfield<',
 '>Smithfield<',
 '>Hoover<',
 '>Murfreesboro<',
 '>Fort Lauderdale<',
 '>San Antonio<',
 '>Indianapolis<',
 '>Portland<',
 '>Great Falls<',
 '>Houston<',
 '>Troy<',
 '>Arlington<',
 '>Huntington Beach<',
 '>New Rochelle<',
 '>Miami<',
 '>Lakewood<',
 '>Rushville<',
 '>Abingdon<',
 '>Lumberton<',
 '>Wahoo<',
 '>A